# 3. Security Aspects and Technical Fundamentals of Blockchain Technology <!-- (*20 Minuten*) -->

## The Idea
The core idea is to provide a system that is through

<!-- Die Kernidee hinter der Blockchain liegt darin ein System zu schaffen das es ermöglicht komplett Dezentral und somit ohne jegliche Intermediäre und damit auch ohne Treuhänder, Transaktionen zwischen Agenten zu ermöglichen.  -->
The core idea behind blockchain technology is to create a decentralized system that allows transactions between parties (agents) without the need for intermediaries or trusted third parties.
<!-- Dabei übernehmen Mathematische Modelle die auf der Berechenbarkeit und Komplexität Kryptographische verschlüsselungen ermgöglichen, die Aufgaben eines Intermediärs und verteilen diese Aufgaben auf alle Agenten die Teil des Systems sind auch wenn diese eine Aufgabe erhalten die nicht ihrer eigenen Transaktion dient. -->
Mathematical models that enable cryptographic encryption based on computability and complexity take on the tasks of an intermediary. The system itself distribute these tasks to all agents in the system, even if they receive a task that is not related to their own transactions.

It is replaced as follows:
| The Object | Centralized FIAT Object | Blockchain |
|-----------|-----------|-----------|
| The Storage of Value | Bank Account / Physical Wallet | Public + Private Key linked to realised transactions held by all participants in the system |
| Transaction | (Digital) Transaction through trustee (middleman) or physical handover between parties | Publishing that an object, has a new owner by its current owner to the whole system through a cryptographic signature |


### 3.1.1 Cryptographic Hash Functions
Because the concept of blockchains is built on different aspects of computation theory, cryptography, and distributed computing, we will start with the core aspects that form a blockchain and begin with the *cryptographic hash functions*.

This is a more uncommon approach in science, but we want to first define what properties our hash functions must hold so they can be used in common blockchains, and we will understand the "Why?" later when everything comes together.

#### Computation
**Computational Property 1: *Input Flexibility***
  * The input can be any string of any size.

**Computational Property 2: *Fixed-Sized Output***
  * The function produces a fixed-sized output.
  * For this discussion, a 256-bit output size is assumed, but the property holds for any sufficiently large output size.

**Computational Property 3: *Efficient Computability***
  * The hash function is efficiently computable, meaning that for a given input string, the output can be determined in a reasonable amount of time.
  * Technically, computing the hash of an n-bit string should have a running time of O(n).


#### Cryptography
**Cryptographic Property 1: *Collision Resistance***

**Definition:** A hash function \( H \) is said to be collision resistant if it is infeasible to find two values, \( x \) and \( y \), such that \( x \neq y \), yet \( H(x) = H(y) \).

*Why we need this?* 
  * We need collision resistance to ensure that it is infeasible to find two different inputs that produce the same hash output.
  * This is crucial for maintaining data integrity and preventing forgery.

*Example:* 
  * When a transaction is created, it is hashed to produce a transaction ID.
  * If an attacker could find two different transactions that produce the same hash (collision), they could potentially replace a legitimate transaction with a fraudulent one without being detected.
  * This would undermine the integrity of the blockchain.

**Cryptographic Property 2: *Hiding***

**Definition:** A hash function \( H \) is said to be hiding if, when a secret value \( r \) is chosen from a probability distribution that has high min-entropy, then, given \( H(r \parallel x) \), it is infeasible to find \( x \).

*Why we need this?* 
  * We need the hiding property to ensure that given a hash output, it is infeasible to determine the original input.
  * This is essential for preserving the confidentiality of the input data.

*Example:* 
  * The hiding property ensures that the nonce and other block header data cannot be easily deduced from the hash output.
  * Maintains the difficulty and security of the mining process.

**Cryptographic Property 3: *Puzzle Friendliness***

**Definition:** A hash function \( H \) is said to be puzzle friendly if for every possible \( n \)-bit output value \( y \), if \( k \) is chosen from a distribution with high min-entropy, then it is infeasible to find \( x \) such that \( H(k \| x) = y \) in time significantly less than \( 2^n \).

*Why we need this?* 
  * Puzzle friendliness ensures that solving the hash function requires a brute-force search.
  * Prevents shortcuts and maintains the security of cryptographic protocols.

*Example:*
  * In Bitcoin mining:
    * Miners must find a nonce such that the hash of the block header concatenated with the nonce produces a hash with a certain number of leading zeros.
    * Puzzle friendliness ensures there are no shortcuts to finding this nonce.
    * Maintains the difficulty and security of the mining process.


### 3.1.2 Hash Pointers
The next thing we need are so called Hash Pointers. Imagine a Linked List

In [ ]:
from linked_list import LinkedList

ll = LinkedList()
ll.append("Beta")
ll.append("Alpha")
ll.append("Delta")
ll.append("Gamma")

ll.print_list()

* **Problem: Ensuring Static Linkage of Values**
  * We need to ensure that values are statically linked together.
  * We want to make sure that we cannot change any value in the middle of the linked list without changing every value before the changed one.

* **Solution: Hash Pointers**
  * Hash Pointers connect a hash of the previous node's data to the current node.
  * Any change in the data of a node will affect the hash, and thus the hash pointer in the next node.
  * This creates a chain reaction, ensuring data integrity.

* **Visualization:**
  * Imagine a linked list where each node contains:
    * Data
    * Hash Pointer to the previous node's data
  * Changing the data in any node will:
    * Change the hash of that node
    * Change the hash pointer in the next node
    * Continue affecting all subsequent nodes


In [ ]:
from linked_list import HashLinkedList

hll = HashLinkedList()
hll.append("Beta")
hll.append("Alpha")
hll.append("Delta")
hll.append("Gamma")
hll.append("Zeta")
hll.append("Omega")

print("Original List:")
hll.print_list()
hll.visualize()


In [ ]:
hll.manipulate_data("Delta", "DeltaModified")
print("\nList after manipulating 'Delta':")
hll.print_list()
hll.visualize()

This structure called *tamper-evident log* is secure against manipulation and hold the following properties:

* Tamper Evidence: Any change in the data can be detected.
*Integrity: The integrity of the data is maintained as long as the hash pointers are intact.

However, this structure alone is not sufficient for a full-fledged blockchain with transactions due to the following reasons:

* Efficiency in Verification: Verifying the integrity of a single block requires traversing the entire chain from the head to the block in question. This can be inefficient for large blockchains.
* Proof of Membership: Proving that a particular transaction exists in the blockchain is not straightforward and requires traversing the entire chain.
* Proof of Nonmembership: Proving that a particular transaction does not exist is even more complex and inefficient.


Merkle trees address these inefficiencies by providing a more structured and efficient way to verify data integrity and membership.


In [ ]:
# https://en.wikipedia.org/wiki/Merkle%E2%80%93Damg%C3%A5rd_construction

### 3.1.3 Merkle Trees

* Efficient Verification: In a Merkle tree, each leaf node represents a data block (e.g., a transaction), and each non-leaf node represents the hash of its child nodes. The root of the tree is a hash that represents the entire dataset. Verifying the integrity of a single block only requires a logarithmic number of hash computations (log(n)), making it much more efficient than traversing the entire chain.

In [5]:
from merkle_tree import merkleTree, visualizeTree

transactions = ['transaction1', 'transaction2', 'transaction3', 'transaction4']
tree = merkleTree()
tree.makeTreeFromArray(transactions)
tree.calculateMerkleRoot()

print("Merkle Root:", tree.getMerkleRoot())
print("\nTree Structure:")
visualizeTree(tree.root)


Merkle Root: 0cf77e26eb4a27047852cf39e3868c7a69ff1109acb9e799ba422d3ac350fb97

Tree Structure:
.: 0cf77e26eb4a27047852cf39e3868c7a69ff1109acb9e799ba422d3ac350fb97
    L: 830b7f324af3ff2b2a47cf7cf5ac825fe8d8034294ea8e8f853a4d3fada893fc
        L: bde4693e55a336ff81ab238ce20cae1dd9c8ba03b9b8f43963f5569bf3cf5229
        R: 4beace8bdcf9b5b74630eaee2e7f501180e46025ca89b05e7e041fbe953d817a
    R: 882ed7f97314e753d0084ea24c386afd3cbd0fb34b3744bfb81d9d3a925cc6e6
        L: 293755ab6384e02d9202d483f2f0250100d786e75fdab1b6f3925b2800ece3cb
        R: 2435dc0372e12b3f7684fb7093fbe6f6dee79dbff96cc28b1687839ef526e02f


### Transaction Validation

**L:** `830b7f324af3ff2b2a47cf7cf5ac825fe8d8034294ea8e8f853a4d3fada893fc`

**Should be:**

- **Transaction 1:** `bde4693e55a336ff81ab238ce20cae1dd9c8ba03b9b8f43963f5569bf3cf5229`
- **+**
- **Transaction 2:** `4beace8bdcf9b5b74630eaee2e7f501180e46025ca89b05e7e041fbe953d817a`

**Result:**

- **Expected:** `830b7f324af3ff2b2a47cf7cf5ac825fe8d8034294ea8e8f853a4d3fada893fc`


In [6]:
from hash_functions import sha256_hash

def verify_merkle_tree():
        txn1_hash = 'bde4693e55a336ff81ab238ce20cae1dd9c8ba03b9b8f43963f5569bf3cf5229'
        txn2_hash = '4beace8bdcf9b5b74630eaee2e7f501180e46025ca89b05e7e041fbe953d817a'
        expected = '830b7f324af3ff2b2a47cf7cf5ac825fe8d8034294ea8e8f853a4d3fada893fc'
        
        combined_hash = sha256_hash(txn1_hash + txn2_hash)

        print(f"Combined hash: {combined_hash}")
        print(f"Expected: {expected}")
        assert combined_hash == expected, f"Hash mismatch: {combined_hash} != {expected}"


verify_merkle_tree()


Combined hash: 830b7f324af3ff2b2a47cf7cf5ac825fe8d8034294ea8e8f853a4d3fada893fc
Expected: 830b7f324af3ff2b2a47cf7cf5ac825fe8d8034294ea8e8f853a4d3fada893fc


We can also prove that the negation is valid.

In [7]:
def verify_merkle_tree():
        txn1_hash = 'bde4693e55a336ff81ab238ce20cae1dd9c8ba03b9b8f43963f5569bf3cf5228'
        txn2_hash = '4beace8bdcf9b5b74630eaee2e7f501180e46025ca89b05e7e041fbe953d817a'
        expected = '830b7f324af3ff2b2a47cf7cf5ac825fe8d8034294ea8e8f853a4d3fada893fc'
        
        
        combined_hash = sha256_hash(txn1_hash + txn2_hash)

        print(f"Combined hash: {combined_hash}")
        print(f"Expected: {expected}")
        assert combined_hash == expected, f"Hash mismatch: {combined_hash} != {expected}"


verify_merkle_tree()


Combined hash: ed16d3014e104c1dea31d8cf9d137daf0231f620b7c39bc291957f15e3ebda4d
Expected: 830b7f324af3ff2b2a47cf7cf5ac825fe8d8034294ea8e8f853a4d3fada893fc


AssertionError: Hash mismatch: ed16d3014e104c1dea31d8cf9d137daf0231f620b7c39bc291957f15e3ebda4d != 830b7f324af3ff2b2a47cf7cf5ac825fe8d8034294ea8e8f853a4d3fada893fc

<!-- Erweiterungs-Angriff (Extension-Attack) -->

<img src="../_static/images/merkle_tree.png">


Important Practical properties Merkle Tree vs. Hashed linked lists:

* Scalability: Merkle trees scale well with the size of the dataset. As the number of transactions grows, the depth of the tree increases logarithmically, maintaining efficient verification times.

* Concise Proofs: Both membership and nonmembership proofs are concise and efficient, making them suitable for large datasets.

* Parallelism: Merkle trees allow for parallel computation of hashes, which can speed up the process of building and verifying the tree.

#### Proof of Membership and Nonmembership

* Proof of Membership: To prove that a particular transaction is part of the blockchain, you only need to provide the transaction, its hash, and the hashes of its sibling nodes up to the root. This proof is concise and can be verified in logarithmic time.

* Proof of Nonmembership: In a sorted Merkle tree, proving nonmembership involves showing the path to the items just before and after the position where the transaction would be. This proof is also efficient and can be verified in logarithmic time.

### 3.1.5 UTXO



In [ ]:
# dump a utxo block

### 3.1.5 Digital Signatures

#### Elliptic Curve Digital Signature Algorithm (ECDSA)

#### Secp256k1

### 3.1.6 Identities

### 3.1.7 Consensus Algorithm

#### Which transactions are the valid ones? (Mining)